In [64]:
#note I had to install astroquery; I installed with  $conda install -c conda-forge astroquery
from astroquery.gaia import Gaia
tables = Gaia.load_tables(only_names=True)
for table in tables:
    print(table.name)
meta = Gaia.load_table('gaiadr2.gaia_source')
for column in meta.columns:
    print(column.name)
    
query1 = """SELECT 
TOP 4096
ra,
ra_error,
dec,
dec_error,
parallax,
parallax_error,
pmra,
pmra_error,
pmdec,
pmdec_error,
phot_g_mean_mag,
phot_bp_mean_mag,
phot_rp_mean_mag,
bp_rp,
bp_g,
g_rp,
radial_velocity,
radial_velocity_error
FROM gaiadr2.gaia_source
WHERE parallax_over_error > 10
AND parallax >= 15 AND parallax <= 50
AND radial_velocity IS NOT null
ORDER BY parallax
"""

job = Gaia.launch_job(query1)

gaia_data = job.get_results()

gaia_data.write('datain/gaia_data.fits', True)

print(f"first star {gaia_data[0]}")



INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
external.apassdr9
external.gaiadr2_geometric_distance
external.gaiaedr3_distance
external.galex_ais
external.ravedr5_com
external.ravedr5_dr5
external.ravedr5_gra
external.ravedr5_on
external.sdssdr13_photoprimary
external.skymapperdr1_master
external.skymapperdr2_master
external.tmass_xsc
public.hipparcos
public.hipparcos_newreduction
public.hubble_sc
public.igsl_source
public.igsl_source_catalog_ids
public.tycho2
public.dual
tap_config.coord_sys
tap_config.properties
tap_schema.columns
tap_schema.key_columns
tap_schema.keys
tap_schema.schemas
tap_schema.tables
gaiaedr3.gaia_source
gaiaedr3.agn_cross_id
gaiaedr3.commanded_scan_law
gaiaedr3.dr2_neighbourhood
gaiaedr3.frame_rotator_source
gaiaedr3.allwise_best_neighbour
gaiaedr3.allwise_neighbourhood
gaiaedr3.apassdr9_best_neighbour
gaiaedr3.apassdr9_join
gaiaedr3.apassdr9_neighbourhood
gaiae

In [65]:
#convert the downloaded .fits file into the format we need


import os
in_file_path = os.path.join(os.getcwd(), 'datain/').replace("\\","/")
out_file_path = os.path.join(os.getcwd(), 'dataout/').replace("\\","/")

taskfile = open('jupyter_convert.task', 'w')
taskfile.write("return {\n")
taskfile.write("\t{\n")
taskfile.write("\t\tType = 'ReadFitsTask',\n")
taskfile.write("\t\tInFileOrFolderPath = '" + in_file_path + "',\n")
taskfile.write("\t\tOutFileOrFolderPath='"+ out_file_path + "',\n")
taskfile.write("\t\tSingleFileProcess = false,\n")
taskfile.write("\t\tThreadsToUse = 6\n",)
taskfile.write("\t}\n")
taskfile.write("}\n")
taskfile.close()

#create construct octree task
octree_taskfile = open('jupyter_octree_convert.task', 'w')
octree_taskfile.write("return {\n")
octree_taskfile.write("\t{\n")
octree_taskfile.write("\t\tType = 'ConstructOctreeTask',\n")
octree_taskfile.write("\t\tInFileOrFolderPath = '" + out_file_path +"',\n")
octree_taskfile.write("\t\tOutFileOrFolderPath = '" + out_file_path +"octree/',\n")
octree_taskfile.write("\t\tMaxDist = 5000,\n")
octree_taskfile.write("\t\tMaxStarsPerNode = 50000,\n")
octree_taskfile.write("\t\tSingleFileInput = false,\n")
octree_taskfile.write("\t}\n")
octree_taskfile.write("}\n")
octree_taskfile.close()

In [66]:
#run task runner to convert fits file for openspace
OPENSPACE_PATH = "c:/os/OpenSpace-0.16.0/"
os.system(OPENSPACE_PATH + "bin/TaskRunner.exe --task jupyter_convert.task")
os.system(OPENSPACE_PATH + "bin/TaskRunner.exe --task jupyter_octree_convert.task")

0

In [67]:
#create openspace .asset 
file = open("jupyter_gaia.asset", "w")
file.write("local assetHelper = asset.require('util/asset_helper')\n")
file.write("local dataFolder = asset.localResource('dataout/octree')\n")
file.write("local node = {\n")
file.write("\tIdentifier = 'jupyter-gaia-sample',\n")
file.write("\tRenderable = {\n")
file.write("\t\tType = 'RenderableGaiaStars',\n")
file.write("\t\tRenderOption = 'Motion',\n")
file.write("\t\tFile = dataFolder .. '/',\n")
file.write("\t\tFileReaderOption = 'StreamOctree',\n",)
file.write("\t\tTexture = asset.localResource('./halo.png'),\n")
file.write("\t\tMagnitudeBoost = 25,\n")
file.write("\t\tLuminosityMultiplier = 80,\n")
file.write("\t\tSigma = 0.5,\n")
file.write("\t\tColorMap = asset.localResource('./charity_bv2rgb.cmap'),\n")
file.write("\t},\n")
file.write("\tGUI={Name='Jupyter Gaia',Path='/Milky Way/Gaia'}\n")
file.write("}\n\n")
file.write("assetHelper.registerSceneGraphNodesAndExport(asset, {node})")
file.close()

In [62]:
#connect to openspace and pause time
from websocket import create_connection
import json
url = "localhost:4682"
ws_endpoint = f"ws://localhost:4682/websocket"
ws = create_connection(ws_endpoint)
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.time.setPause",
                              "arguments": ["true"],
                              "return": False}})
ws.send(message)

131

In [63]:
#add the asset
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.asset.add",
                              "arguments": [os.getcwd() + "/jupyter_gaia.asset"],
                              "return": False}})
ws.send(message)

#refresh menu
message = json.dumps({"topic": 4,
                  "type": "luascript",
                  "payload": {"function": "openspace.setPropertyValueSingle",
                              "arguments": ['Modules.CefWebGui.Reload', None],
                              "return": False}})
ws.send(message)

168